In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline

import csv

#!pip install request
import requests # library to handle requests

#!pip install beautifulsoup4 
from bs4 import BeautifulSoup

#!pip install lxml

#!pip install html5lib

#!conda install -c conda-forge geopy --yes # module Niminatim
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium

!pip install pgeocode # Postal code geocoding and distance calculations https://pypi.org/project/pgeocode/
import pgeocode

from collections import OrderedDict

import math
from difflib import SequenceMatcher

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    conda-4.6.7                |           py36_0         869 KB  conda-forge
    openssl-1.1.1b             |       h14c3975_0         4.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.8 MB

The following packages will be UPDATED:

  conda                                        4.6.4-py36_0 --> 4.6.7-py36_0
  openssl                              1.1.1a-h14c3975_1000 --> 1.1.1b-h14c3975_0



conda-4.6.7          | 869 KB    | ##################################### | 100% 
openssl-1.1.1b       | 4.0 MB    | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executi

# Coursera Course - Applied Data Science Capstone
## Housing prices - Hamburg

### 1. Introduction

#### 1.1 Description of the problem


#### 1.2 Description of the data


##### i) Boroughs and neighborhoods

##### ii) Post codes

##### iv) Venues for each Post code?, Neighborhood? or Boroughs

##### v) Transport, Train stations, S-Bahn stations, Subway stations, Bus stations

##### vi) Housing prices, archiv and current 

####  Scrap wikipedia - Boroughs and Neighborhoods of Hamburg

In [2]:
wiki_borough_link = "https://de.wikipedia.org/wiki/Liste_der_Bezirke_und_Stadtteile_Hamburgs#Stadtteile"

In [3]:
r= requests.get(wiki_borough_link)
soup = BeautifulSoup(r.text,'lxml')

In [4]:
#print(soup.prettify())

In [5]:
#Definition of the dataframe 'Boroughs and Neighborhoods'
# define columns
col_names = ['neighborhood','borough','area','population', 'coord_nord','coord_east'] 
# initiate dataframe
df_neighborhoods = pd.DataFrame(columns=col_names)

In [6]:
# Function to transform coordinates from geographic to decimal
def convert_decimal(coordinate):
    coord = ''.join(coordinate.split())[:-2].upper()
    coord_d, coord_m, coord_s = coord.replace("°","_").replace("′","_").split('_')                            
    coord = "{0:.5f}".format(float(coord_d) + (float(coord_m) / 60) + (float(coord_s) / 3600))
    return coord

In [7]:
table = soup.find_all('table', class_='wikitable sortable')
# iterate over it
wikidatasaved = []
for item in table[1].find('tbody').find_all('tr'):
    wikidata = []
    for data in item.find_all('td'):
        wikidata.append(data.text.strip())
    
    #print(wikidata[0:8])
    if len(wikidata) > 0:
            b, wikidata[0] = wikidata[0].split("!")
            b, wikidata[3] = wikidata[3].split("!")
            coord_n, coord_e = wikidata[6].split(",")
            coord_n = convert_decimal(coord_n)
            coord_e = convert_decimal(coord_e)
            df_neighborhoods = df_neighborhoods.append({'neighborhood': wikidata[0],'borough': wikidata[2], 'area' : wikidata[3], 'population' : wikidata[4], 'coord_nord':coord_n, 'coord_east':coord_e}, ignore_index=True)
            

In [8]:
df_neighborhoods.sort_values(by=['neighborhood']).head()

,neighborhood,borough,area,population,coord_nord,coord_east
81,Allermöhe,Bergedorf,"8,6",1361,53.48611,10.15417
45,Alsterdorf,Hamburg-Nord,"3,2",15.031,53.61083,10.01306
76,Altengamme,Bergedorf,"15,6",2246,53.42472,10.26250
98,Altenwerder,Harburg,"6,7",3,53.50694,9.91778
19,Altona-Altstadt,Altona,"2,8",22.946,53.54889,9.94778


####  Scrap hamburg.de - post codes of hamburg

In [9]:
plz_link = "https://www.hamburg.de/postleitzahlen/"
r= requests.get(plz_link)
soup = BeautifulSoup(r.text,'lxml')
#print(soup.prettify())

In [10]:
# define columns
col_plz_names = ['neighborhood','post_codes'] 
# initiate dataframe
df_neigh_plz = pd.DataFrame(columns=col_plz_names)

In [11]:
table = soup.find_all('table')
# iterate over it
#datasaved = []
for item in table[0].find('tbody').find_all('tr'):
    plzdata = []
    for data in item.find_all('td'):
        if data.text.strip() != '':
            plzdata.append(data.text.strip())
    df_neigh_plz = df_neigh_plz.append({'neighborhood': plzdata[0],'post_codes': ",".join(plzdata[1:len(plzdata)])}, ignore_index=True)

df_neigh_plz.head()

,neighborhood,post_codes
0,Allermöhe,"21035,21037,21113"
1,Alsterdorf,"20251,22297,22335,22337"
2,Altengamme,"21029,21039"
3,Altenwerder,21129
4,Altona-Altstadt,"20359,22765,22767,22769"


In [12]:
# search double entries and join them, the 
myindex = []
for index, row in df_neigh_plz.iterrows():
    if row[0].find('Forts') > 1:
        a,b = row[0].split(" ")
        myindex.append(index)
        for ind,row1 in df_neigh_plz.iterrows():
            if row1[0] == a:
                #print(a)
                #print(row[0],":",row[1])
                #print(row1[0],':',row1[1])
                row1[1] = row1[1] + ',' + row[1]
                #print(row1[1])
                
                break
df_neigh_plz.drop(df_neigh_plz.index[myindex], inplace=True)

In [13]:
#Verification number of neighborhoods
df_neigh_plz.shape

(104, 2)

In [14]:
# fucntion to evaluate the similarity of two strings
def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

df_neigh = pd.merge(df_neighborhoods, df_neigh_plz, on='neighborhood',how='left')

for index, row in df_neigh.iterrows():
    #print(row[0],row[6])
    if isinstance(row[6],str) != True:
        for index_plz, row_plz in df_neigh_plz.iterrows():
            if similar(row[0],row_plz[0]) > 0.79:
                #print('(**)',row[0],row[6],"<-",row_plz[0]," ",row_plz[1])
                row[6] = row_plz[1]

In [15]:
df_neigh.head()

,neighborhood,borough,area,population,coord_nord,coord_east,post_codes
0,Hamburg-Altstadt,Hamburg-Mitte,"2,4",2305,53.55000,10.00000,"20095,20099,20457"
1,HafenCity,Hamburg-Mitte,"2,2",3627,53.54111,10.00028,"20457,20539"
2,Neustadt,Hamburg-Mitte,"2,3",12.719,53.55194,9.98556,"20354,20355,20359,20457,20459"
3,St. Pauli,Hamburg-Mitte,"2,5",22.501,53.55694,9.96389,"20354,20355,20357,20359,22767,22769"
4,St. Georg,Hamburg-Mitte,"2,4",11.055,53.55500,10.01222,"20095,20097,20099"


#### Scrap hauses from immowelt.de

In [17]:
imo_link = "https://www.immowelt.de/liste/hamburg/haeuser/kaufen?sort=relevanz"
r= requests.get(imo_link)
soup = BeautifulSoup(r.text,'lxml')

In [18]:
# number of pages to scrap
tot_pages = []
sitenum = soup.find(id='pnlPaging')#, class_='site-numbers')
for item in sitenum.find_all('a'):
    if item.text != '':
        tot_pages.append(item.text.strip())
numpages = max(list(map(int,tot_pages)))

In [19]:
numpages = 2

#### scrap all immo page with selling hauses

In [20]:
imo_pages = []
for i in range(1,numpages+1):
    imo_link = "https://www.immowelt.de/liste/hamburg/haeuser/kaufen?sort=relevanz&cp=" + str(i)
    r= requests.get(imo_link)
    soup = BeautifulSoup(r.text,'lxml')
    imo_pages.append(soup)
    print('Reading page number:' + str(i))
print('Scrap finished!')

Reading page number:1
Reading page number:2
Scrap finished!


In [21]:
# Crate the dataframe
df_imo_cols = ['oid','stateid','link'] 
# initiate dataframe
df_imo = pd.DataFrame(columns=df_imo_cols)
# scrap data
for impage in imo_pages:
    for item in impage.find_all('div', class_='js-object'):# listitem_wrap blickfang'):
        df_imo = df_imo.append({'oid': item['data-oid'] , 'stateid': item['data-estateid'] , 'link' : 'https://www.immowelt.de/expose/' + item['data-oid'] }, ignore_index=True)
            
print(df_imo.shape)
df_imo.head()

(48, 3)


,oid,stateid,link
0,2N65P42,54547557,https://www.immowelt.de/expose/2N65P42
1,2P5JK46,55723524,https://www.immowelt.de/expose/2P5JK46
2,2PFK546,55709230,https://www.immowelt.de/expose/2PFK546
3,2P3A445,55675138,https://www.immowelt.de/expose/2P3A445
4,2PT4S43,55631962,https://www.immowelt.de/expose/2PT4S43


In [22]:
# Drop duplicates
df_imo = df_imo.drop_duplicates(subset='oid', keep="last")
df_imo = df_imo.reset_index(drop=True)
print(df_imo.shape)
df_imo.head()

(40, 3)


,oid,stateid,link
0,2PT4S43,55631962,https://www.immowelt.de/expose/2PT4S43
1,2PSJW44,55669273,https://www.immowelt.de/expose/2PSJW44
2,2N4AJ44,54608131,https://www.immowelt.de/expose/2N4AJ44
3,2PSBK44,55657785,https://www.immowelt.de/expose/2PSBK44
4,2P9FJ44,55656872,https://www.immowelt.de/expose/2P9FJ44


In [23]:
# Crate dataframe haus fcats
df_hause_cols = ['address','post_code','land_area','house_area','rooms','price','features'] 
# initiate dataframe
df_hause = pd.DataFrame(columns=df_hause_cols)

# Scrap anouncement of each haus
for index, row in df_imo.iterrows():
    print('Scraping page ',index,':',row[2])
    imo_hause = row[2] #'https://www.immowelt.de/expose/2PCFE43'
    r= requests.get(imo_hause)
    soup = BeautifulSoup(r.text,'lxml')
    
    
    
    address = soup.find('div', class_='location').find('span').text
    plz = address.split()[0]
    
    itlist = []
    item = soup.find('div', class_='hardfacts clear')
    for it in item.find_all('div'):
        itlist.append(it)
    
    land_area = itlist[6].text
    land_area = land_area.split()[0]
    
    house_area = itlist[2].text
    house_area = house_area.split()[0]
    
    rooms = itlist[4].text
    rooms = rooms.split()[0]
    
    price = itlist[0].text
    price = price.split()[0]
    
    try:
        features = soup.find('div', class_='merkmale').text
            
    except:
        features = ''
        
    df_hause = df_hause.append({'address' : address, 'post_code' : plz, 'land_area' : land_area, 'house_area' : house_area, 'rooms' : rooms, 'price' : price, 'features' : features}, ignore_index=True)

Scraping page  0 : https://www.immowelt.de/expose/2PT4S43
Scraping page  1 : https://www.immowelt.de/expose/2PSJW44
Scraping page  2 : https://www.immowelt.de/expose/2N4AJ44
Scraping page  3 : https://www.immowelt.de/expose/2PSBK44
Scraping page  4 : https://www.immowelt.de/expose/2P9FJ44
Scraping page  5 : https://www.immowelt.de/expose/2PUA544
Scraping page  6 : https://www.immowelt.de/expose/2PPV444
Scraping page  7 : https://www.immowelt.de/expose/2P6AZ43
Scraping page  8 : https://www.immowelt.de/expose/2P6XX43
Scraping page  9 : https://www.immowelt.de/expose/2PCFE43
Scraping page  10 : https://www.immowelt.de/expose/2P4TL42
Scraping page  11 : https://www.immowelt.de/expose/2NK8L4Z
Scraping page  12 : https://www.immowelt.de/expose/2N3WH4M
Scraping page  13 : https://www.immowelt.de/expose/2MMYQ44
Scraping page  14 : https://www.immowelt.de/expose/2N27V4X
Scraping page  15 : https://www.immowelt.de/expose/2NXXU4R
Scraping page  16 : https://www.immowelt.de/expose/2N65P42
Scrapin

In [24]:
df_hause.head()

,address,post_code,land_area,house_area,rooms,price,features
0,22041 Hamburg (Wandsbek),22041,590,238,7,979.000,"Balkon, Terrasse, Garten, Bad mit Wanne, Gäste..."
1,21035 Hamburg (Allermöhe),21035,250,136,5,465.000,"Terrasse, Bad mit Wanne, Gäste WC, Einbauküche..."
2,21079 Hamburg (Moorburg),21079,k.A.,30,k.A.,69.000,"provisionsfrei, Terrasse, Bad mit Wanne, Neubau"
3,"22457 Hamburg (Schnelsen), Königskinderweg 74",22457,486,117,4,516.800,"provisionsfrei, Terrasse, Stellplatz, Bad mit ..."
4,"22457 Hamburg (Schnelsen), Königskinderweg 74",22457,237,121,4,474.900,"provisionsfrei, Terrasse, Stellplatz, Bad mit ..."


#### Clean dataframe -> Price, k.A. values, etc

In [25]:
# remove point as thousands symbol
df_hause['price'] = df_hause['price'].str.replace('.','')
df_hause['land_area'] = df_hause['land_area'].str.replace('.','')
df_hause['house_area'] = df_hause['house_area'].str.replace('.','')

# change decimal separator from ',' to '.'
df_hause['land_area'] = df_hause['land_area'].str.replace(',','.')
df_hause['house_area'] = df_hause['house_area'].str.replace(',','.')
df_hause['rooms'] = df_hause['rooms'].str.replace(',','.')

In [26]:
# remove not valid entries as rows with k.A.
df_hause = df_hause[df_hause.post_code != 'k.A.']
df_hause = df_hause[df_hause.land_area != 'k.A.']
df_hause = df_hause[df_hause.house_area != 'k.A.']
df_hause = df_hause[df_hause.rooms != 'k.A.']

# convert to float
df_hause['land_area'] = pd.to_numeric(df_hause['land_area'], downcast='float')
df_hause['house_area'] = pd.to_numeric(df_hause['house_area'], downcast='float')
df_hause['rooms'] = pd.to_numeric(df_hause['rooms'], downcast='float')
df_hause['price'] = pd.to_numeric(df_hause['price'], downcast='float')

In [28]:
df_hause.head()

,address,post_code,land_area,house_area,rooms,price,features
0,22041 Hamburg (Wandsbek),22041,590.0,238.0,7.0,979000.0,"Balkon, Terrasse, Garten, Bad mit Wanne, Gäste..."
1,21035 Hamburg (Allermöhe),21035,250.0,136.0,5.0,465000.0,"Terrasse, Bad mit Wanne, Gäste WC, Einbauküche..."
3,"22457 Hamburg (Schnelsen), Königskinderweg 74",22457,486.0,117.0,4.0,516800.0,"provisionsfrei, Terrasse, Stellplatz, Bad mit ..."
4,"22457 Hamburg (Schnelsen), Königskinderweg 74",22457,237.0,121.0,4.0,474900.0,"provisionsfrei, Terrasse, Stellplatz, Bad mit ..."
5,22159 Hamburg (Farmsen-Berne),22159,248.0,125.0,5.0,635000.0,"provisionsfrei, Terrasse, Garten, Bad mit Wann..."


In [29]:
df_hause.describe()

,land_area,house_area,rooms,price
count,37.000000,37.000000,37.000000,37.000
mean,681.189209,162.338379,5.378378,800243.000
std,467.507935,65.363838,1.478576,644982.625
min,139.000000,63.000000,4.000000,359000.000
25%,300.000000,121.199997,4.500000,485900.000
50%,590.000000,137.500000,5.000000,585000.000
75%,870.000000,180.000000,5.000000,930000.000
max,2200.000000,350.000000,10.000000,3999000.000


####  Create a map of Hamburg including markers for the neighborhoods

In [29]:
# create map of Hamburg
address = '22041, Hamburg'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
map_hamburg = folium.Map(location=[latitude, longitude], width=800, height=600, zoom_start=11)
#folium.CircleMarker([float(latitude), float(longitude)], radius=4, popup=label, color='blue', fill=True, fill_color='#3186cc', fill_opacity=0.6, parse_html=False).add_to(map_hamburg) 
#map_hamburg

In [28]:
# add markers to map
for lat, lng, label in zip(df_neigh['coord_nord'], df_neigh['coord_east'], df_neigh['neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([float(lat), float(lng)],radius=4,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.6,parse_html=False).add_to(map_hamburg)  
#map_hamburg

#### Analisys Foursquare
Find the 100 Venues of each postal code ina 1000 m radius

In [16]:
CLIENT_ID = 'TK52SZTRXS01EHZ5BC2401INEK1FEH0CY2B3H3KOGLLE3PT0' # your Foursquare ID
CLIENT_SECRET = 'RQI32Z1S54PMZH43RJ420DP3KELRLR21TAD3BFBEIB4MTQFR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
print('Your credentails -> CLIENT_ID: ' + CLIENT_ID + ' CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails -> CLIENT_ID: TK52SZTRXS01EHZ5BC2401INEK1FEH0CY2B3H3KOGLLE3PT0 CLIENT_SECRET:RQI32Z1S54PMZH43RJ420DP3KELRLR21TAD3BFBEIB4MTQFR


#### Check the 100 top venues of each neighborhood

In [17]:
# function to search the 100 top venues
def getNearbyVenues(names, latitudes, longitudes, radius, LIMIT):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        
        # analysis post code
        try:
            pp = results[0]['venue']['location']['postalCode']
        except:
            pp = float('nan')
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            pp,
            v['venue']['location']['formattedAddress'],
            v['venue']['categories'][0]['name']) for v in results])
            
            

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['neighborhood',
                             'coord_nord',
                             'coord_east',
                             'venue',
                             'venue_coord_nord',
                             'venue_coord_east',
                             'venue_post_code',
                             'venue_address',
                             'venue_category'
                             ]
    nearby_venues.dropna()
    
    return(nearby_venues)

In [18]:
hamburg_venues = getNearbyVenues(df_neigh['neighborhood'], df_neigh['coord_nord'], df_neigh['coord_east'], 1000, 100)

Hamburg-Altstadt
HafenCity
Neustadt
St. Pauli
St. Georg
Hammerbrook
Borgfelde
Hamm
Horn
Billstedt
Billbrook
Rothenburgsort
Veddel
Wilhelmsburg
Kleiner Grasbrook
Steinwerder
Waltershof
Finkenwerder
Neuwerk
Altona-Altstadt
Sternschanze
Altona-Nord
Ottensen
Bahrenfeld
Groß Flottbek
Othmarschen
Lurup
Osdorf
Nienstedten
Blankenese
Iserbrook
Sülldorf
Rissen
Eimsbüttel
Rotherbaum
Harvestehude
Hoheluft-West
Lokstedt
Niendorf
Schnelsen
Eidelstedt
Stellingen
Hoheluft-Ost
Eppendorf
Groß Borstel
Alsterdorf
Winterhude
Uhlenhorst
Hohenfelde
Barmbek-Süd
Dulsberg
Barmbek-Nord
Ohlsdorf
Fuhlsbüttel
Langenhorn
Eilbek
Wandsbek
Marienthal
Jenfeld
Tonndorf
Farmsen-Berne
Bramfeld
Steilshoop
Wellingsbüttel
Sasel
Poppenbüttel
Hummelsbüttel
Lemsahl-Mellingstedt
Duvenstedt
Wohldorf-Ohlstedt
Bergstedt
Volksdorf
Rahlstedt
Lohbrügge
Bergedorf
Curslack
Altengamme
Neuengamme
Kirchwerder
Ochsenwerder
Reitbrook
Allermöhe
Billwerder
Moorfleet
Tatenberg
Spadenland
Neuallermöhe
Harburg
Neuland
Gut Moor
Wilstorf
Rönneburg


In [20]:
hamburg_venues.to_pickle('df_hamburg_venues.pkl')

In [30]:
hamburg_venues.shape

(3159, 9)

In [21]:
df = pd.read_pickle('df_hamburg_venues.pkl')

In [22]:
df.head()

,neighborhood,coord_nord,coord_east,venue,venue_coord_nord,venue_coord_east,venue_post_code,venue_address,venue_category
0,Hamburg-Altstadt,53.55000,10.00000,GOOT - Finest Cuts,53.548509,10.000675,20095,"[Depenau 10 (Niedernstr.), 20095 Hamburg, Deut...",Comfort Food Restaurant
1,Hamburg-Altstadt,53.55000,10.00000,Shiso Burger,53.550648,10.001897,20095,"[Bugenhagenstr 23, 20095 Hamburg, Deutschland]",Burger Joint
2,Hamburg-Altstadt,53.55000,10.00000,Passage Kino,53.550708,9.998299,20095,"[Mönckebergstr. 17, 20095 Hamburg, Deutschland]",Indie Movie Theater
3,Hamburg-Altstadt,53.55000,10.00000,Thalia Theater,53.551897,9.998384,20095,"[Alstertor 1, 20095 Hamburg, Deutschland]",Theater
4,Hamburg-Altstadt,53.55000,10.00000,O-ren Ishii,53.548402,9.997628,20095,"[Kleine Reichenstr. 18, 20457 Hamburg, Deutsch...",Vietnamese Restaurant


In [25]:
print('There are {} uniques categories.'.format(len(hamburg_venues['venue_category'].unique())))

There are 299 uniques categories.


In [38]:
hamburg_onehot = pd.get_dummies(hamburg_venues[['venue_category']], prefix="", prefix_sep="")
hamburg_onehot['neighborhood'] = hamburg_venues['neighborhood']


In [42]:
fixed_columns = [hamburg_onehot.columns[-1]] + list(hamburg_onehot.columns[:-1])
hamburg_onehot = hamburg_onehot[fixed_columns]

In [47]:
hamburg_onehot.shape

(3159, 300)

In [56]:
hamburg_grouped = hamburg_onehot.groupby('neighborhood').mean().reset_index()
hamburg_grouped.head()

,neighborhood,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Aquarium,...,Video Store,Vietnamese Restaurant,Volleyball Court,Water Park,Whisky Bar,Wine Bar,Wine Shop,Yoga Studio,Zoo,Zoo Exhibit
0,Allermöhe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
1,Alsterdorf,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.05,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
2,Altengamme,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
3,Altenwerder,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
4,Altona-Altstadt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.01,0.0,0.01,0.01,0.02,0.0,0.0,0.0,0.0


In [64]:
num_top_venues = 5

for hood in hamburg_grouped['neighborhood']:
    #print("----\\\ "+hood+" //----")
    temp = hamburg_grouped[hamburg_grouped['neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    #print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    #print('\n')

In [74]:
# set number of clusters
kclusters = 7

hamburg_grouped_clustering = hamburg_grouped.drop('neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(hamburg_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_
unique, counts = np.unique(kmeans.labels_, return_counts=True)
print (np.asarray((unique, counts)).T)

[[ 0 31]
 [ 1  4]
 [ 2  1]
 [ 3 45]
 [ 4 21]
 [ 5  1]
 [ 6  1]]


#### Extract post codes from GeoJason file of Hamburg 

In [17]:
import json # library to handle JSON files

In [18]:
# read jso from Hamburg with post code polygons
with open('plz-5stellig.geojson') as json_data:
    hamburg_data = json.load(json_data)

In [19]:
#hamburg_data

In [69]:
plz_data = hamburg_data['features']

In [22]:
# define the dataframe columns
column_names = ['postal_code', 'population', 'postal_code_area' ,'polygon'] 

# instantiate the dataframe
df_coordplz = pd.DataFrame(columns=column_names)

In [24]:
for data in plz_data:
    plzn = data['properties']['plz'] 
    population = data['properties']['einwohner']
    postalcodearea = data['properties']['qkm']
    
    plz_polygon = data['geometry']['coordinates'][0]
    
    df_coordplz = df_coordplz.append({'postal_code': plzn,
                                          'population': population,
                                          'postal_code_area': postalcodearea,
                                          'polygon': plz_polygon}, ignore_index=True)

In [62]:
list_plz = []
for nn in df_neigh['post_codes']:
    for t in nn.split(','):
        list_plz.append(t)
        
list_plz = pd.DataFrame(list(set(list_plz)),columns=['postal_code'])
list_plz.shape

(105, 1)

In [66]:
# select the post code of hamburg
df_coordplz_hh = pd.merge(df_coordplz, list_plz, how='right', on=['postal_code'])

In [68]:
df_coordplz_hh.head()

,postal_code,population,postal_code_area,polygon
0,20095,3172,0.759332,"[[9.9911721, 53.5511273], [9.9917439, 53.55136..."
1,20097,1690,1.968448,"[[10.006869500000002, 53.5457261], [10.0076566..."
2,20099,9050,1.588046,"[[9.9973681, 53.5569902], [9.9974811, 53.55712..."
3,20144,11315,0.939591,"[[9.9678144, 53.5731396], [9.9680212, 53.57311..."
4,20146,10469,0.931351,"[[9.9708645, 53.5676491], [9.9751571, 53.56969..."


In [79]:
latitude = float(df_neigh.loc[1, 'coord_nord'])
longitude = float(df_neigh.loc[1, 'coord_east'])
map_hamburg = folium.Map(location=[latitude, longitude], zoom_start=11)
folium.CircleMarker([float(latitude), float(longitude)], radius=6, popup='Center', color='red', fill=True, fill_color='#3186cc', fill_opacity=0.6, parse_html=False).add_to(map_hamburg) 

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([float(lat), float(lng)],radius=4,popup=label,color='blue',fill=True,fill_color='#3186cc',fill_opacity=0.6,parse_html=False).add_to(map_hamburg)  
#map_hamburg